In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge,RidgeCV,LassoCV,Lasso
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_squared_error,root_mean_squared_error
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [47]:
data=pd.read_pickle("data_free_outlier.pkl")

In [48]:
data.head()

,CRIM,ZN,INDUS,CHAS,NX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NX       506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [50]:
X=data.drop("MEDV",axis=1)

In [51]:
y=data["MEDV"]

In [52]:
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=0.3,random_state=10)

In [53]:
numaric_transform=Pipeline(steps=[("imputer",SimpleImputer()),("scaler",StandardScaler())])

In [54]:
tier1=ColumnTransformer(transformers=[("simple_imput",numaric_transform,[0,1,2,3,4,5,6,7,8,9,10,11,12])],remainder='passthrough')

In [55]:
tier2_liner=LinearRegression()

In [56]:
alphas=np.logspace(-4,4,50)

In [57]:
tier_3_Cv_ridge=RidgeCV(alphas=alphas)


In [58]:
tier_4_Cv_lasso=LassoCV(alphas=alphas)

## linear Regression Pipeline

In [59]:
pip1=Pipeline([("Liner_tier1",tier1),
              ("tier2_liner",tier2_liner),])



In [65]:
pip1

Pipeline(steps=[('Liner_tier1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simple_imput',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])])),
                ('tier2_liner', LinearRegression())])

In [60]:
pip1.fit(train_x,train_y)
print("R2 score",pip1.score(test_x,test_y))


R2 score 0.7043035668988956


## Ridge Regression Pipeline

In [61]:
pip2_r=Pipeline([("Liner_tier1",tier1),
              ("tier_3_Cv_ridge",tier_3_Cv_ridge),])

In [66]:
pip2_r

Pipeline(steps=[('Liner_tier1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simple_imput',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])])),
                ('tier_3_Cv_ridge',
                 RidgeCV(alphas=array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095...
       8.28642773e-01, 1.20679264e+00, 1.75751062e+00, 2.55954792e+00,
       3.72759372e+00, 5.42867544e+00, 7.90604321e+00, 1.15139540e+01,
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04])))])

In [62]:
pip2_r.fit(train_x,train_y)
print("R2 score",pip2_r.score(test_x,test_y))

R2 score 0.7035609739461681


## lasso Regression Pipeline

In [63]:
pip3_l=Pipeline([("Liner_tier1",tier1),
              ("tier_4_Cv_lasso",tier_4_Cv_lasso),])

In [67]:
pip3_l

Pipeline(steps=[('Liner_tier1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('simple_imput',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                   10, 11, 12])])),
                ('tier_4_Cv_lasso',
                 LassoCV(alphas=array([1.00000000e-04, 1.45634848e-04, 2.12095089e-04, 3.08884360e-04,
       4.49843267e-04, 6.55128557e-04, 9.54095...
       8.28642773e-01, 1.20679264e+00, 1.75751062e+00, 2.55954792e+00,
       3.72759372e+00, 5.42867544e+00, 7.90604321e+00, 1.15139540e+01,
       1.67683294e+01, 2.44205309e+01, 3.55648031e+01, 5.17947468e+01,
       7.54312006e+01, 1.09854114e+02, 1.59985872e+02, 2.32995181e+02,
       3.39322177e+02, 4.94171336e+02, 7.19685673e+02, 1.04811313e+03,
       1.52641797e+03, 2.22299648e+03, 3.23745754e+03, 4.71486636e+03,
       6.86648845e+03, 1.00000000e+04])))])

In [64]:
pip3_l.fit(train_x,train_y)
print("R2 score",pip3_l.score(test_x,test_y))

R2 score 0.7044676558071759
